# 4: Working With a Cluster

At this point we are hopefully able to connect to a cluster and have a basic understanding of the difference between head-nodes and compute-nodes. In this notebook we will start looking at how to work with a cluster.

First we will talk about the filesystem in order to get an understanding of where we should place and work with our files. Secondly we will cover the concept of "Environment Modules" on the cluster. Then we will look at how to copy files onto the cluster and outline how to edit scripts directly on the cluster.


## 4.1: Filesystem + Storage

Before we start putting files on the cluster we need to know that we have options and what these options are. Broadly speaking, there are three relevant locations for storage on the large Princeton clusters. These are `/home`, `/scratch/gpfs`, and `/tigress`. These storage locations serve different purposes that we must consider when choosing where to put our files.

| **STORAGE OPTIONS** |                                          |
|---------------------|------------------------------------------|
| **`/home`**         |                                          |
| intent:             | Source code and executables.             |
| storage:            | ~20GB                                    |
| back up:            | Backed up daily                          |
| **`/scratch/gpfs`** |                                          |
| intent:             | Intermediate results.                    |
| storage:            | ~500GB storage.                          |
| back up:            | Never backed up.                         |
| **`/tigress`**      |                                          |
| intent:             | Source code, executables, final results. |
| storage:            | ~500GB storage.                          |
| back up:            | Backed up weekly.                        |


As can be seen from the table, the main considerations are amount of storage and back up policy. `/home` and `/tigress` are backed up, but have different storage capacities. `/scratch/gpfs` is not backed up, but provides a lot of storage. Another difference to be aware of is that `/home` and `/scratch/gpfs` are local storage systems, while `/tigress` is a storage system that is mounted across all the clusters. Practically, this means that any files we put in `/tigress` will be available on any of the clusters, while files that go in local storage (`/home` or `/scratch/gpfs`) are only available on that specific cluster.

Note that it is recommended to work on `/scratch/gpfs` when generating intermediate results because these do not need to be backed up and since `/scratch/gpfs` is a local storage system, it is faster than using `/tigress`. Once the final results have been generated, they can be moved to `/tigress` such that they are backed up.

There are some other storage options available such as `/scratch` and `/scratch/network` that serve other purposes. For more information on these options, follow the links below for the relevant cluster. For information on storage options on Nobel, Adroit, and Tigressdata, follow the links below.

**More Information:**

- [Nobel](https://researchcomputing.princeton.edu/systems-and-services/available-systems/nobel)
- [Adroit](https://researchcomputing.princeton.edu/systems-and-services/available-systems/adroit)
- [Tigressdata](https://researchcomputing.princeton.edu/systems-and-services/available-systems/tigressdata)
- [Tiger](https://researchcomputing.princeton.edu/systems-and-services/available-systems/tiger)
- [Della](https://researchcomputing.princeton.edu/systems-and-services/available-systems/della)
- [Perseus](https://researchcomputing.princeton.edu/systems-and-services/available-systems/perseus)

In order to check how much storage we are currently using on a cluster, we use the `checkquota` command in the terminal. This gives us an overview of all the storage options on the cluster and how much of our quota we are currently using for each location. Increases in quota can be requested using this [form](https://forms.rc.princeton.edu/quota/).

### 4.1.1: Filesystem and Storage Stewardship

Having a lot of storage available to us makes it easy to use more than we need. Storage is an expensive resource and it is difficult to provide enough hardware to keep up with the demand of more and more users with larger and larger data. Therefore, as users of these resources, it is important that we clean up and get rid of any data that we don't need. Since files on the system seldom get removed (even after a user has left the university), there will inevitably be a lot of useless data taking up a considerable amount of storage on the system. To do our part, we should take on the following two guidelines:

1. Remove any temporary data and results that we don't need.
2. Compress any final results before storing them on `/tigress`.

This can significantly decrease the amount of useless data in circulation on the clusters.


## 4.2: Environment Modules

All the Princeton clusters use Environment Modules which makes it easier to use software packages. All available software packages are not available by default when you log on to a cluster, but can be "loaded" using Environment Modules. Examples of software packages that are commonly used and must be loaded on the clusters are Matlab and Anaconda. Environment Modules use the `module` command, and all available modules on a cluster can be listed by running the following command:

```bash
module avail
```

This results in a long list of available software packages that we can load and use. In order to load a module we use the command:

```bash
module load <name_of_module>
```

For example, if we want to use the R2018a version of Matlab, we can load it with

```bash
module load matlab/R2018a
```

Now Matlab will be available to use on the cluster with the `matlab` command in the terminal. The following is an overview of the most useful `module` commands:

| **MODULE COMMANDS**              |                                                      |
|----------------------------------|------------------------------------------------------|
| `module avail`                   | list all available modules on the system             |
| `module load <name_of_module>`   | load a module                                        |
| `module list`                    | list all loaded modules                              |
| `module unload <name_of_module>` | unload a module                                      |
| `module purge`                   | unload all modules                                   |
| `module show <name_of_module>`   | show which environment variables the module modifies |


## 4.3: Copying Files to the Cluster

In order to work with code and data on the cluster we need a way of getting the relevant files onto the cluster. For most purposes we can use commands such as `scp`, `ftp`, or `rsync`. If the files we want to move are very large, we might want to use a tool such as [Globus](https://researchcomputing.princeton.edu/services/globus_description).

In this section we will look at how to use `scp` from the command line, but the concepts are similar if you prefer to use `ftp` or `rsync` instead. First we will look at how to copy files from a local machine (personal laptop or Princeton desktop machine) onto the cluster, and then we will look at how to copy files from the cluster down to our local machine.

Copying files with `scp` is done with a single command that has the following syntax:

```bash
scp <source> <target>
```

It takes the file that is located at `<source>` and copies it to the `<target>` location. If we want to copy an entire folder instead, we just need to modify this command as follows

```bash
scp -r <source> <target>
```

When we want to copy files to or from the cluster, the first step is to open up a terminal window on our **local machine** and perform one of the following steps.

**From Local Machine to Cluster:**

Copying files from our local machine to the cluster means that `<source>` is a file or folder on our local machine and `<target>` is a location on the cluster. `<source>` can be given as either a relative path or total path on our system. `<target>` is here a remote system which means that we must specify both the target's hostname and relative path or total path. With this in mind, the command we want to use will take the following form:

```bash
scp -r /path/on/local/machine/ <your_puid>@<hostname>.princeton.edu:/path/on/cluster
```

For example, if we have a folder `~/Desktop/project/` on our local machine that we want to copy onto `/scratch/gpfs` on Della, we would use the following command

```bash
scp -r ~/Desktop/project/ <your_puid>@della.princeton.edu:/scratch/gpfs/<your_puid>
```

**From Cluster to Local Machine:**

When copying files from the cluster to our local machine, it means that `<source>` is a file or folder on the cluster, and `<target>` is a location on our local machine. The syntax is the same as above, but with `<source>` and `<target>` swapped:

```bash
scp -r <your_puid>@<hostname>.princeton.edu:/path/on/cluster /path/on/local/machine/
```

If we want to copy a folder `/scratch/gpfs/<your_puid>/project/` from Della to the Desktop on our local machine, we would use the following command

```bash
scp -r <your_puid>@della.princeton.edu:/scratch/gpfs/project/ ~/Desktop/
```

**NOTE:** In both cases, we are issuing the `scp` commands from a terminal on our local machine, not on the cluster.


## 4.4: Editing Scripts on the Cluster

If we are running code on the cluster and want to change something in a script, we could make the relevant changes to the script on our local machine and then copy the script onto the cluster. This, however, soon becomes tedious. Having to copy files to the cluster every time we want to make a change means a lot of copying if we are making a lot of changes. A better way to do this is to edit the scripts directly on the cluster through the terminal. We do this using one of the terminal text editors.

Text editors are just what the name suggests, programs to edit text. Being able to do this directly in the terminal is very powerful and almost essential if we are doing a lot of work on remote machines. There are several text editors available on the Princeton clusters. Two text editors that are incredibly powerful, but come with higher learning curves are `vim` and `emacs`. Alternatives that are more beginner friendly are `micro` and `nano`.

If you are planning to do a lot of work in the terminal, you are best served choosing either `vim` or `emacs`. If your interest is more limited, `micro` or `nano` might be more suitable. I any case, choose one that you like and stick with it. Find tutorials online on how to use your text editor of choice and you will be editing your scripts directly on the cluster in no time.